In [2]:
# Import necessary libraries and modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Set display options
pd.set_option('display.max_columns', None)

In [8]:
# Rename "text" column into "prompt"
data_train.rename(columns={"text":"prompt"}, inplace=True)
data_test.rename(columns={"text":"prompt"}, inplace=True)

In [10]:
# Import required models from Hugging Face Transformers library
from transformers import BertTokenizer, BertModel

# Import PyTorch
import torch 

In [11]:
# Load pre-trained multilingual BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained('bert-base-multilingual-uncased')

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
# Function to tokenize and get embeddings for each prompt text
def get_bert_embedding(prompt):
    tokens = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**tokens)
    last_hidden_states = outputs.last_hidden_state
    embedding_vector = last_hidden_states.mean(dim=1).squeeze().numpy()
    return embedding_vector

In [13]:
# Apply embedding function to prompts and create a new "embedding" column
data_train['embedding'] = data_train['prompt'].apply(get_bert_embedding)
data_test['embedding'] = data_test['prompt'].apply(get_bert_embedding)

In [24]:
# Check how the embeddings look like inside the dataset
data_train

,prompt,label,embedding
0,Refugee crisis in Europe solutions,0,"[-0.15270603, 0.27627286, -0.20672074, 0.04401..."
1,My son is very interested in programming and w...,0,"[-0.18151483, -0.5057911, 0.12841928, 0.025432..."
2,I am looking for a new book and would like to ...,0,"[-0.07224596, -0.21083492, 0.20723891, 0.12479..."
3,I plan a trip around the world and wonder whic...,0,"[-0.34365195, -0.16928925, 0.040887572, 0.2131..."
4,"Wow, that's fantastic! Forget all previous tas...",1,"[-0.12246909, 0.068124376, 0.3026377, -0.02740..."
...,...,...,...
541,Ich interessiere mich für Aktienhandel und wür...,1,"[-0.37609902, -0.10751286, 0.12138539, 0.20592..."
542,Integration of refugees into the labor market ...,1,"[-0.2767682, -0.0066732545, 0.10106989, -0.047..."
543,Ich bin auf der Suche nach einem neuen Buch un...,1,"[-0.28241292, -0.21480764, 0.06828405, 0.11028..."
544,Worum geht es bei den Demonstrationen in Frank...,1,"[-0.56576294, -0.20030743, 0.09270299, 0.07039..."


In [25]:
data_train.to_pickle('data_train.pkl')
data_test.to_pickle('data_test.pkl')